In [1]:
import logging
import subprocess
import json
import argparse
from typing import Dict, List
import shutil
import ray
from ray.util import ActorPool
import os
import sys
import gc

from rcnn_iwp_inference import load_model, get_metadata, run_inference


In [2]:
@ray.remote(num_cpus=1, memory=4 * 1024 * 1024 * 1024)
class ImageInferenceActor:
    def __init__(self, model_ref, metadata_ref):
        self.model = model_ref
        self.metadata = metadata_ref

    def process_image(self, image_path, threshold, output_folder, worker_id):
        try:
            print(f"Worker {worker_id} running inference on {image_path}")
            
            image_file = os.path.basename(image_path)
            
            # Create unique temporary output file using worker_id and image name
            temp_output_file = f"output_{worker_id}_{image_file}.jpg"
            
            # Run inference with custom output path
            coco_mask_metadata = run_inference(
                self.model, 
                image_path, 
                self.metadata, 
                threshold=threshold, 
                output_file=temp_output_file
            )
            
            # Create output file path
            output_file = os.path.join(output_folder, f"{image_file.split('.')[0]}_masked.jpg")
            
            # Copy the output image to the output folder
            shutil.copy(temp_output_file, output_file)
            os.remove(temp_output_file)
            
            # Force garbage collection to free memory
            gc.collect()
            
            return {"file": image_file, "metadata": coco_mask_metadata, "output_file": output_file}
        except Exception as e:
            print(f"Error processing image {image_path}: {e}")
            # Force garbage collection even on error
            gc.collect()
            return {"file": os.path.basename(image_path), "error": str(e)}

In [3]:
# Load the model

model_file = "./model_weights.pth"
model = load_model("configs/mask_rcnn_vitdet.py", "configs/datasets/iwp.py", model_file)
metadata = get_metadata("iwp_test", "data/iwp/iwp_test.json")


/home/mohanar2/.conda/envs/clowderenv/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/home/mohanar2/.conda/envs/clowderenv/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
INFO:detectron2.checkpoint.detection_checkpoint:[DetectionCheckpointer] Loading from ./model_weights.pth ...
INFO:fvcore.common.checkpoint:[Checkpointer] Loading from ./model_weights.pth ...


In [4]:
# Initialize Ray

ray.init(address="auto", ignore_reinit_error=True)


2025-06-16 15:41:57,547	INFO worker.py:1654 -- Connecting to existing Ray cluster at address: 192.168.100.25:6379...
2025-06-16 15:41:57,561	INFO worker.py:1832 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.16
Ray version:,2.43.0
Dashboard:,http://127.0.0.1:8265


(ImageInferenceActor pid=48330) /home/mohanar2/.conda/envs/clowderenv/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
(ImageInferenceActor pid=48330)   check_for_updates()
(ImageInferenceActor pid=48323) /home/mohanar2/.conda/envs/clowderenv/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
(ImageInferenceActor pid=48323)   check_for_updates()


In [5]:
#  Add to store
model_ref = ray.put(model)
metadata_ref = ray.put(metadata)


In [6]:
output_folder_name = f"output_images_masked"
output_folder = os.path.join(os.getcwd(), output_folder_name)
os.makedirs(output_folder, exist_ok=True)

In [9]:
dataset_folder = 'test_images'
dataset_folder = os.path.join(os.getcwd(), dataset_folder)
image_files = [os.path.join(dataset_folder, f) for f in os.listdir(dataset_folder)]

print(image_files)

['/home/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_289_Polygon_1.jpg', '/home/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_301_Polygon_1.jpg', '/home/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_320_Polygon_1.jpg', '/home/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_337_Polygon_2.jpg', '/home/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_280_Polygon_1.jpg', '/home/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_292_Polygon_1.jpg', '/home/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_308_Polygon_1.jpg', '/home/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_342_Polygon_2.jpg', '/home/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_310_Polygon_1.jpg', '/home/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_303_Polygon_1.jpg', '/home/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_291_Pol

In [10]:
NUM_ACTORS = 4
actors = [ImageInferenceActor.remote(model_ref, metadata_ref) for _ in range(NUM_ACTORS)]
actors_pool = ActorPool(actors)

In [11]:
# Create task inputs with unique IDs
tasks = [
    (image_path, 0.6, output_folder, i) 
    for i, image_path in enumerate(image_files)
]

# Process all images using the actor pool
results = list(actors_pool.map(lambda actor, task: actor.process_image.remote(*task), tasks))

(ImageInferenceActor pid=48323) Worker 2 running inference on /home/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_320_Polygon_1.jpg


(ImageInferenceActor pid=48322) /home/mohanar2/.conda/envs/clowderenv/lib/python3.10/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1708025569588/work/aten/src/ATen/native/TensorShape.cpp:3549.)
(ImageInferenceActor pid=48322)   return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
(ImageInferenceActor pid=48321) /home/mohanar2/.conda/envs/clowderenv/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1. [repeated 3x across cluster]
(ImageInferenceActor pid=48321)   check_for_updates() [repeated 3x across cluster]


(ImageInferenceActor pid=48331) Worker 4 running inference on /home/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_280_Polygon_1.jpg [repeated 4x across cluster]
(ImageInferenceActor pid=48331) Worker 8 running inference on /home/mohanar2/pdg-extractors/rcnn_iwp_inference_slurm/test_images/FID_310_Polygon_1.jpg [repeated 4x across cluster]


In [10]:
print(results)

[{'file': 'FID_289_Polygon_1.jpg', 'metadata': [{'id': 1, 'category_id': 1, 'bbox': [401.5414123535156, 180.4000244140625, 140.86947631835938, 121.52325439453125], 'area': 17118.91796875, 'score': 0.9910733103752136}, {'id': 2, 'category_id': 1, 'bbox': [242.68704223632812, 72.04177856445312, 113.76043701171875, 154.21917724609375], 'area': 17544.041015625, 'score': 0.9909319877624512}, {'id': 3, 'category_id': 1, 'bbox': [126.1694564819336, 68.78707885742188, 121.7298355102539, 115.40927124023438], 'area': 14048.751953125, 'score': 0.9878501296043396}, {'id': 4, 'category_id': 1, 'bbox': [74.00088500976562, 206.77825927734375, 160.08566284179688, 147.02947998046875], 'area': 23537.3125, 'score': 0.9877816438674927}, {'id': 5, 'category_id': 1, 'bbox': [247.4387969970703, 349.73675537109375, 138.92955017089844, 102.25531005859375], 'area': 14206.2841796875, 'score': 0.9875988364219666}, {'id': 6, 'category_id': 1, 'bbox': [264.09869384765625, 230.13491821289062, 151.97235107421875, 130